<font face="Calibri">
<br>
<font size="6"> <b> AVO Obsarviewory:<b> </font>
<font size="5"> <b> routine InSAR processing tool for Alaska volcanoes<b> </font></b>
<br>
<br>
<font size="5"> <b> Current Notebook: routine_processing<b> </font></b>

<br>
<br>
<br>
<font size="4"> 
In this notebook, for each asf project that can be updated, new interferogram requests are submitted to ASF. Then corresponding volcano-project-wise timeseries analysis will begin, incuding download of data and run of MintPy.
<br>
<br>
This notebook reads in <code>data/asflist.pkl</code> and <code>data/volclist.pkl</code> for reference. If those files do not exist, please try the <code>insar_list_init</code> Notebook.</b>
<br>
<br>
This Notebook is part of <a href="https://github.com/uafgeotools/avo_obsarviewtory" target="_blank">AVO Obsarviewtory</a>.  


</font>
<hr>

In [ ]:
import pickle
import avo_insar_functions
import hyp3_sdk as sdk
from pathlib import Path

class asf_area:
    def __init__(self, asf_name, path, frame, last_date):
        self.asf_name = asf_name
        self.path = path
        self.frame = frame
        self.last_date = last_date
    def update_date(self,new_date):
        self.last_date = new_date
with open('data/asflist.pkl', 'rb') as f:
    asf_list = pickle.load(f)

class volcano_area:
    def __init__(self, volc_name, filter_dates, ul, lr, path, frame, asf_name, run_flag):
        self.volc_name = volc_name
        self.filter_dates = filter_dates
        self.ul = ul
        self.lr = lr
        self.path = path
        self.frame = frame
        self.asf_name = asf_name
        self.run_flag = run_flag
    def update_flag(self, new_flag):
        self.run_flag = new_flag
with open('data/volclist.pkl', 'rb') as f:
    volcano_list = pickle.load(f)


flags = [] # this variable records if a project should be updated
hyp3 = sdk.HyP3('https://hyp3-avo.asf.alaska.edu', prompt=True, username='ycheng', password='earthdata+1S')
for asf_project in asf_list:
    asf_flag = avo_insar_functions.update_asf_project(asf_project.path,asf_project.frame,asf_project.asf_name,hyp3) # this function checks updates AND submit requests to ASF
    flags.append( asf_flag ) # save the status: can be updated?
    asf_project.update_date(asf_flag)


watch_name = 'name' # this variable records an asf_name
for i in range(len(flags)):
    for volcano in volcano_list:
        if volcano.asf_name == asf_list[i].asf_name:
            volcano.update_flag( flags[i] )
    if flags[i] == 1:
        watch_name = asf_list[i].asf_name # always save the lastly-submitted project for watch
if watch_name != 0:
    jobs = hyp3.find_jobs(name=watch_name)
    jobs = hyp3.watch(jobs) # wait until asf has processed all submitted requests


for volcano in volcano_list: 
    if volcano.run_flag == 1: # run all processings that can be updated
        
        print('########CURRENT PROJECT########')
        project_name = volcano.volc_name + str(volcano.path)
        analysis_directory = Path.cwd() / 'processings' / project_name
        mintpy_directory = analysis_directory / 'MintPy'
        print(f"analysis_directory: {analysis_directory}")

        gdal_command = avo_insar_functions.avo_insar_download( hyp3 , volcano.asf_name , analysis_directory , volcano.filter_dates)
        full_scene = analysis_directory/"full_scene.tif"
        if full_scene.exists():
            full_scene.unlink()
        !{gdal_command}
        image_file = f"{analysis_directory}/raster_stack.vrt"
        !gdalbuildvrt -separate $image_file -overwrite $full_scene

        avo_insar_functions.avo_insar_crop(image_file, volcano.ul, volcano.lr, analysis_directory)

        mintpy_config = avo_insar_functions.avo_insar_run( analysis_directory , mintpy_directory )
        !smallbaselineApp.py --dir {mintpy_directory} {mintpy_config}
        clean_command = f"rm -rf {analysis_directory}/S1*"
        !clean_command

with open('data/asflist.pkl', 'wb') as f:  # update local record
    pickle.dump(asf_list, f)

<br>  
<br>  
<br>  

<div class="alert alert-warning" role="alert">
    <div class="row vertical-align">
        <div class="col-xs-1 text-center">
            <i class="fa fa-exclamation-triangle fa-2x"></i>
        </div>
        <div class="col-xs-11">
                <strong>The following cells are not part of a routine processing.</strong>                   
        </div>   
    </div> 
</div>

In [22]:
import pickle
import avo_insar_functions
import hyp3_sdk as sdk
from pathlib import Path

class asf_area:
    def __init__(self, asf_name, path, frame, last_date):
        self.asf_name = asf_name
        self.path = path
        self.frame = frame
        self.last_date = last_date
    def update_date(self,new_date):
        self.last_date = new_date
with open('data/asflist.pkl', 'rb') as f:
    asf_list = pickle.load(f)

class volcano_area:
    def __init__(self, volc_name, filter_dates, ul, lr, path, frame, asf_name, run_flag):
        self.volc_name = volc_name
        self.filter_dates = filter_dates
        self.ul = ul
        self.lr = lr
        self.path = path
        self.frame = frame
        self.asf_name = asf_name
        self.run_flag = run_flag
    def update_flag(self, new_flag):
        self.run_flag = new_flag
with open('data/volclist.pkl', 'rb') as f:
    volcano_list = pickle.load(f)

flags = [] # this variable records if a project should be updated
hyp3 = sdk.HyP3('https://hyp3-avo.asf.alaska.edu', prompt=True, username='ycheng', password='earthdata+1S')
for asf_project in asf_list:
    asf_flag = avo_insar_functions.update_asf_project(asf_project.path,asf_project.frame,asf_project.asf_name,hyp3)
    flags.append( asf_flag ) # save the status: is updated?
    asf_project.update_date(asf_flag)

Upated ASF project submitted. Your new subset contains 2 more pairs.
Upated ASF project submitted. Your new subset contains 2 more pairs.
Upated ASF project submitted. Your new subset contains 2 more pairs.
